# Pregunta 8

## De las características de altura, peso y talla; realice su propio dataset determinando cuál será su clase. Realice el cálculo de la entropía y ganancia de información.

### R.-

**What is Entropy?**

Entropy is used for a lot of things in data science. For example, entropy can be used to build classification trees which are used to classify things or datasets. Entropy is also the basis of something called 'mutual information' which quantifies the relationship between two things. Entropy is also the basis of Relative Entropy aka ‘The Kullback-Liebler distance’ and Cross-Entropy which show up all over the place including fancy dimension reduction algorithms like t-SNE and UMAP. What these three things have in common is that they all use entropy or something derived from it to quantify similarities and differences so let’s learn how entropy quantifies similarities and differences.

Entropy is the measure of uncertainty of a variable. The more uncertain it is, the higher the entropy is.

> *¿Qúe es la entropía?*
>
> *La entropía se utiliza para muchas cosas en la ciencia de datos. Por ejemplo, la entropía se puede utilizar para crear árboles de clasificación que se utilizan para clasificar cosas o conjuntos de datos. La entropía también es la base de algo llamado "información mutua" que cuantifica la relación entre dos cosas. La entropía también es la base de la entropía relativa, también conocida como "la distancia Kullback-Liebler" y la entropía cruzada, que aparecen por todas partes, incluidos sofisticados algoritmos de reducción de dimensiones como t-SNE y UMAP. Lo que estas tres cosas tienen en común es que todas usan la entropía o algo derivado de ella para cuantificar similitudes y diferencias, así que aprendamos cómo la entropía cuantifica similitudes y diferencias.*
>
> *La entropía es la medida de incertidumbre de una variable. Cuanto más incierto es, mayor es la entropía.*

Fórmula para la entropía para una variable:

$p(x) \log_2 (p(x))$

Fórmula para la entropía para un sistema

$H(x)=-\sum_{x=0}^{n}{p(x) \log_2(p(x))}$

Creando un dataset

La clase de este dataset será la talla, así tendremos

In [46]:
import numpy as np
from scipy.stats import multivariate_normal

# Definir la media y matriz de covarianza
mean = [1.7, 70, 52]  # Altura peso, y talla promedio

cov = [[0.09, 0.15, 0.05],
       [0.15, 3.4, 0.1],
       [0.05, 0.1, 4]]  # Matriz de covarianza

# Generar datos
data = multivariate_normal.rvs(mean, cov, size=100)

# Crear DataFrame
import pandas as pd
df = pd.DataFrame(data, columns=['Altura', 'Peso', 'Talla'])

df['Talla'] = np.round(df['Talla']).astype(int)

# df['']

In [47]:
print(df)

      Altura       Peso  Talla
0   2.291224  71.573518     52
1   1.331398  70.097056     52
2   0.803125  69.567036     52
3   1.454176  70.897794     48
4   1.440983  69.029468     50
..       ...        ...    ...
95  1.560757  67.519436     47
96  2.030334  70.175749     49
97  1.571238  73.524368     52
98  1.679529  72.825874     52
99  1.302324  67.214814     54

[100 rows x 3 columns]


In [48]:
df.describe()

,Altura,Peso,Talla
count,100.000000,100.000000,100.000000
mean,1.671530,69.936435,52.150000
std,0.304940,1.744611,2.001893
min,0.803125,65.879686,47.000000
25%,1.440794,68.483216,51.000000
50%,1.680115,69.938527,52.000000
75%,1.908499,71.218195,53.000000
max,2.307982,73.798667,58.000000


La clase será la talla, lo que haremos será hacer un LabelEncoder. Tomamos como referencia las tallas de pantalón de https://www.confeccionesgala.com/equivalencia-de-tallas/.

Función de mapeo

In [49]:
# Definir una función para mapear las tallas a categorías
def map_talla(talla):
    if talla < 44:
        return 'XS'
    elif 44 <= talla < 48:
        return 'S'
    elif 48 <= talla < 52:
        return 'M'
    elif 52 <= talla < 56:
        return 'L'
    elif 56 <= talla < 60:
        return 'XL'
    elif 60 <= talla < 64:
        return 'XXL'
    elif talla >= 68:
        return 'XXXL'
    else:
        return 'unknown'

In [50]:
# Aplicar la función al DataFrame
df['Talla'] = df['Talla'].apply(map_talla)

print(df)


      Altura       Peso Talla
0   2.291224  71.573518     L
1   1.331398  70.097056     L
2   0.803125  69.567036     L
3   1.454176  70.897794     M
4   1.440983  69.029468     M
..       ...        ...   ...
95  1.560757  67.519436     S
96  2.030334  70.175749     M
97  1.571238  73.524368     L
98  1.679529  72.825874     L
99  1.302324  67.214814     L

[100 rows x 3 columns]


Así la clase es **Categoria**

Función para calcular la entropía

In [51]:
from math import log2

def calcular_entropia(clase):
    valores, cuentas = np.unique(clase, return_counts=True)
    probabilidades = cuentas / len(clase)
    entropia = -np.sum([p * log2(p) for p in probabilidades if p > 0])
    return entropia

Como son valores continuos los del dataset, calculamos la entropía por intervalos usando *binarización*

In [52]:
import numpy as np
import pandas as pd


def imprimir_tabla(df, target_col, bins=5):
    
    # Discretizar las columnas continuas
    for col in df.columns:
        if col != target_col and df[col].dtype != 'object':
            df[col] = pd.cut(df[col], bins=bins)
    
    entropia_total = calcular_entropia(df[target_col])

    print(f"\t\tTotal\tXS\tS\tM\tL\tXL\tXXL\tXXXL\tEntropia")
        
    xs = df[target_col].value_counts().get('XS', 0)
    s = df[target_col].value_counts().get('S', 0)
    m = df[target_col].value_counts().get('M', 0)
    l = df[target_col].value_counts().get('L', 0)
    xl = df[target_col].value_counts().get('XL', 0)
    xxl = df[target_col].value_counts().get('XXL', 0)
    xxxl = df[target_col].value_counts().get('XXXL', 0)
    
    columns_text_1 = f'{xs}\t{s}\t{m}\t{l}\t{xl}\t{xxl}\t{xxxl}'
    
    print(f"Pareja\t\t{len(df)}\t{columns_text_1}\t{entropia_total:.6f}")
    print("")

    for col in df.columns:
        if col == target_col:
            continue
        
        print(f"{col}")
        print(f"Intervalo\t\tXS\tS\tM\tL\tXL\tXXL\tXXXL\tEntropia")
        
        for valor in df[col].unique():
            subset = df[df[col] == valor]
            entropia_cond = calcular_entropia(subset[target_col])
            cuenta_xs = subset[target_col].value_counts().get('XS', 0)
            cuenta_s = subset[target_col].value_counts().get('S', 0)
            cuenta_m = subset[target_col].value_counts().get('M', 0)
            cuenta_l = subset[target_col].value_counts().get('L', 0)
            cuenta_xl = subset[target_col].value_counts().get('XL', 0)
            cuenta_xxl = subset[target_col].value_counts().get('XXL', 0)
            cuenta_xxxl = subset[target_col].value_counts().get('XXXL', 0)

            columns_text_2 = f'{cuenta_xs:.4f}\t{cuenta_s:.4f}\t{cuenta_m:.4f}\t{cuenta_l:.4f}\t{cuenta_xl:.4f}\t{cuenta_xxl:.4f}\t{cuenta_xxxl:.4f}'
            
            print(f"{valor}")
            print(f"\t\t{len(subset)}\t{columns_text_2}\t{entropia_cond:.6f}")
            # print(f"\t\t{len(subset)}\t{columns_text_2}\t{entropia_cond:.6f}")
            
        
        print("")

In [53]:
imprimir_tabla(df, 'Talla', bins=5)

		Total	XS	S	M	L	XL	XXL	XXXL	Entropia
Pareja		100	0	2	29	63	6	0	0	1.294258

Altura
Intervalo		XS	S	M	L	XL	XXL	XXXL	Entropia
(2.007, 2.308]
		13	0.0000	0.0000	3.0000	9.0000	1.0000	0.0000	0.0000	1.140116
(1.104, 1.405]
		22	0.0000	0.0000	6.0000	13.0000	3.0000	0.0000	0.0000	1.351687
(0.802, 1.104]
		2	0.0000	0.0000	1.0000	1.0000	0.0000	0.0000	0.0000	1.000000
(1.405, 1.706]
		28	0.0000	1.0000	10.0000	16.0000	1.0000	0.0000	0.0000	1.335238
(1.706, 2.007]
		35	0.0000	1.0000	9.0000	24.0000	1.0000	0.0000	0.0000	1.170185

Peso
Intervalo		XS	S	M	L	XL	XXL	XXXL	Entropia
(70.631, 72.215]
		29	0.0000	0.0000	7.0000	20.0000	2.0000	0.0000	0.0000	1.130738
(69.047, 70.631]
		32	0.0000	1.0000	9.0000	22.0000	0.0000	0.0000	0.0000	1.042599
(67.463, 69.047]
		25	0.0000	1.0000	10.0000	11.0000	3.0000	0.0000	0.0000	1.602740
(65.872, 67.463]
		6	0.0000	0.0000	1.0000	5.0000	0.0000	0.0000	0.0000	0.650022
(72.215, 73.799]
		8	0.0000	0.0000	2.0000	5.0000	1.0000	0.0000	0.0000	1.298795



La ganancia de información se calcula de la siguiente manera:

$ Ganancia(S,A) = Entropia(S)−\sum_{v \in Valores(A)}{( \frac{∣Sv∣}{∣S∣} \times Entropia(S_v))}$

Donde:

- $S$ es el conjunto de datos completo.
- $A$ es el atributo para el cual estamos calculando la ganancia.
- $Valores(A)$ son los distintos valores que toma el atributo $A$.
- $Sv$​ es el subconjunto de S donde el atributo $A$ tiene el valor $v$.
- $∣Sv​∣$ es el número de instancias en $Sv$​.
- $Entropía(S)$ es la entropía del conjunto completo.
- $Entropía(Sv​)$ es la entropía del subconjunto $Sv$​.

In [54]:
def calcular_ganancia(df, atributo, target_col):
    entropia_total = calcular_entropia(df[target_col])
    valores, cuentas = np.unique(df[atributo], return_counts=True)
    entropia_ponderada = 0.0
    for valor, cuenta in zip(valores, cuentas):
        subset = df[df[atributo] == valor]
        entropia_subset = calcular_entropia(subset[target_col])
        entropia_ponderada += (cuenta / len(df)) * entropia_subset
    ganancia = entropia_total - entropia_ponderada
    return ganancia


In [55]:
def calcular_ganancias(df, target_col):
    atributos = [col for col in df.columns if col != target_col]
    ganancias = {}
    for atributo in atributos:
        ganancia = calcular_ganancia(df, atributo, target_col)
        ganancias[atributo] = ganancia
    return ganancias


ganancias = calcular_ganancias(df, 'Talla')
for atributo, ganancia in ganancias.items():
    print(f"Ganancia de Información para '{atributo}': {ganancia:.6f}")


Ganancia de Información para 'Altura': 0.045240
Ganancia de Información para 'Peso': 0.089122
